In [18]:
import pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dtt
import statsmodels.api as sm
from pylab import mpl

In [52]:
mpl.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
class Data(object):
    """本类用于从mysql中提取相应条件的数据"""
    def __init__(self, sql, cur, args=None):
        self.sql = sql
        self.cur = cur
        self.args = args
        self.data = Data.get_data(self)

    def __str__(self):
        return str(self.data)

    __repr__ = __str__

    def get_data(self):
        _ = self.cur.execute(self.sql, self.args)
        data = self.cur.fetchall()
        return data

    def select_col(self, col):
        return [d[col] for d in self.data]

In [4]:
db = pymysql.connect("localhost", "root", "root", "strategy1", charset="utf8")
cur = db.cursor()


In [60]:
sql0 = r"""select dt, rate from tb_rate where term = 10 and bond_type ='国债'"""
data0 = Data(sql0, cur)
dt = np.array(data0.select_col(0))
rate = np.array(data0.select_col(1))
fig, axes = plt.subplots(3, 1, figsize=(12, 6), sharex="all", gridspec_kw={'height_ratios':[3, 1.5, 1.5]})
# 图1，十年国债到期收益率
axes[0].spines["top"].set_color('none')
axes[0].spines["right"].set_color("none")
axes[0].xaxis.set_ticks_position("bottom")
axes[0].yaxis.set_ticks_position("left")
axes[0].plot(dt, rate, label="十年国债收益率")
axes[0].spines["bottom"].set_position(('data', 3))
axes[0].legend(fontsize=15)
# 图2，国债发行冲击
sql1 = r"""select dt, delta from tb_sec_delta where code0 regexp '[:alnum:]{2}00.*' and term >=3"""
data1 = Data(sql1, cur)
dt1 = np.array(data1.select_col(0))
delta1 = np.array(data1.select_col(1))
axes[1].bar(dt1, delta1, label="续发国债冲击（BP)")
axes[1].set_ylim(-10, 10)
axes[1].legend(fontsize=10, loc="upper left")
# 图3，国开债发行冲击
sql2 = r"""select dt, delta from tb_sec_delta 
where code0 regexp '[:alnum:]{2}02.*' and term = 10
and delta is not null"""
data2 = Data(sql2, cur)
dt2 = np.array(data2.select_col(0))
delta2 = np.array(data2.select_col(1))
axes[2].bar(dt2, delta2, label="续发国开债冲击（BP）")
axes[2].set_ylim(-10, 10)
axes[2].legend(fontsize=10, loc="upper left")

In [61]:
plt.savefig(r"C:\Users\Administrator\Desktop\lll.png")
fig.show()


In [39]:
help(axes[1].set_ylim)

Help on method set_ylim in module matplotlib.axes._base:

set_ylim(bottom=None, top=None, emit=True, auto=False, *, ymin=None, ymax=None) method of matplotlib.axes._subplots.AxesSubplot instance
    Set the data limits for the y-axis
    
    .. ACCEPTS: (bottom: float, top: float)
    
    Parameters
    ----------
    bottom : scalar, optional
        The bottom ylim (default: None, which leaves the bottom
        limit unchanged).
        The bottom and top ylims may be passed as the tuple
        (`bottom`, `top`) as the first positional argument (or as
        the `bottom` keyword argument).
    
    top : scalar, optional
        The top ylim (default: None, which leaves the top limit
        unchanged).
    
    emit : bool, optional
        Whether to notify observers of limit change (default: True).
    
    auto : bool or None, optional
        Whether to turn on autoscaling of the y-axis. True turns on,
        False turns off (default action), None leaves unchanged.
    
  

In [24]:
help(plt.subplots)

Help on function subplots in module matplotlib.pyplot:

subplots(nrows=1, ncols=1, sharex=False, sharey=False, squeeze=True, subplot_kw=None, gridspec_kw=None, **fig_kw)
    Create a figure and a set of subplots.
    
    This utility wrapper makes it convenient to create common layouts of
    subplots, including the enclosing figure object, in a single call.
    
    Parameters
    ----------
    nrows, ncols : int, optional, default: 1
        Number of rows/columns of the subplot grid.
    
    sharex, sharey : bool or {'none', 'all', 'row', 'col'}, default: False
        Controls sharing of properties among x (`sharex`) or y (`sharey`)
        axes:
    
            - True or 'all': x- or y-axis will be shared among all
              subplots.
            - False or 'none': each subplot x- or y-axis will be
              independent.
            - 'row': each subplot row will share an x- or y-axis.
            - 'col': each subplot column will share an x- or y-axis.
    
        Wh